# Build a song recommender system

In [ ]:
!pip install turicreate

In [ ]:
import os.path
from google.colab import drive

# mount Google Drive to /content/drive/My Drive/
if os.path.isdir("/content/drive/My Drive/"):
  print("Google Drive already mounted")
else:
  drive.mount('/content/drive')

Google Drive already mounted


In [ ]:
import os.path
import urllib.request
import tarfile
import zipfile
import gzip
from shutil import copy

def fetch_remote_datafile(filename, remote_url):
  if os.path.isfile("./" + filename):
    print("already have " + filename + " in workspace")
    return
  print("fetching " + filename + " from " + remote_url + "...")
  urllib.request.urlretrieve(remote_url, "./" + filename)

def cache_datafile_in_drive(filename):
  if os.path.isfile("./" + filename) == False:
    print("cannot cache " + filename + ", it is not in workspace")
    return
  
  data_drive_path = "/content/drive/My Drive/Colab Notebooks/"
  if os.path.isfile(data_drive_path + filename):
    print("" + filename + " has already been stored in Google Drive")
  else:
    print("copying " + filename + " to " + data_drive_path)
    copy("./" + filename, data_drive_path)
  

def load_datafile_from_drive(filename, remote_url=None):
  data_drive_path = "/content/drive/My Drive/Colab Notebooks/"
  if os.path.isfile("./" + filename):
    print("already have " + filename + " in workspace")
  elif os.path.isfile(data_drive_path + filename):
    print("have " + filename + " in Google Drive, copying to workspace...")
    copy(data_drive_path + filename, ".")
  elif remote_url != None:
    fetch_remote_datafile(filename, remote_url)
  else:
    print("error: you need to manually download " + filename + " and put in drive")
    
def extract_datafile(filename, expected_extract_artifact=None):
  if expected_extract_artifact != None and (os.path.isfile(expected_extract_artifact) or os.path.isdir(expected_extract_artifact)):
    
    print("files in " + filename + " have already been extracted")
  elif os.path.isfile("./" + filename) == False:
    print("error: cannot extract " + filename + ", it is not in the workspace")
  else:
    extension = filename.split('.')[-1]
    if extension == "zip":
      print("extracting " + filename + "...")
      data_file = open(filename, "rb")
      z = zipfile.ZipFile(data_file)
      for name in z.namelist():
          print("    extracting file", name)
          z.extract(name, "./")
      data_file.close()
    elif extension == "gz":
      print("extracting " + filename + "...")
      if filename.split('.')[-2] == "tar":
        tar = tarfile.open(filename)
        tar.extractall()
        tar.close()
      else:
        data_zip_file = gzip.GzipFile(filename, 'rb')
        data = data_zip_file.read()
        data_zip_file.close()
        extracted_file = open('.'.join(filename.split('.')[0:-1]), 'wb')
        extracted_file.write(data)
        extracted_file.close()
    elif extension == "tar":
      print("extracting " + filename + "...")
      tar = tarfile.open(filename)
      tar.extractall()
      tar.close()
    elif extension == "csv":
      print("do not need to extract csv")
    else:
      print("cannot extract " + filename)
      
def load_cache_extract_datafile(filename, expected_extract_artifact=None, remote_url=None):
  load_datafile_from_drive(filename, remote_url)
  extract_datafile(filename, expected_extract_artifact)
  cache_datafile_in_drive(filename)
  
load_cache_extract_datafile("song_data.sframe.zip", "song_data.sframe", "https://d3c33hcgiwev3.cloudfront.net/dIBuXeIiEemm5A4ynZyB2A_749ef60351624a8d83beedd5ef23367e_song_data.sframe.zip?Expires=1582329600&Signature=O30-yejeeiJkSvfIZvD~tAKnG4f6kQJb5fa~3h2CTpw4dxbUoUOX8fIpIyjaKlApahk5AsL1-45RakOgO1CmyDgW5RdZzGCUXloTUcRDYLlg0Q2lyy0ImpqlXL~lvhEMn-XTrtR6pQ64aYInGIgHFpDpDc5zZMKYypdMQc-rIPE_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A")

already have song_data.sframe.zip in workspace
files in song_data.sframe.zip have already been extracted
song_data.sframe.zip has already been stored in Google Drive


In [ ]:
import turicreate

# Load some music data

In [ ]:
song_data = turicreate.SFrame('./song_data.sframe/')

# Explore our data

In [ ]:
song_data

user_id,song_id,listen_count,title,artist
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOAKIMP12A8C130995,1,The Cove,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Paco De Lucia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBXHDL12A81C204C0,1,Stronger,Kanye West
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBYHAJ12A6701BF1D,1,Constellations,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODACBL12A8C13C273,1,Learn To Fly,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODDNQT12A6D4F5F7E,5,Apuesta Por El Rock 'N'Roll ...,Héroes del Silencio
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODXRTY12AB0180F3B,1,Paper Gangsta,Lady GaGa
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFGUAY12AB017B0A8,1,Stacked Actors,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFRQTD12A81C233C0,1,Sehr kosmisch,Harmonia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOHQWYZ12A6D4FA701,1,Heaven's gonna burn youreyes ...,Thievery Corporationfeat. Emiliana Torrini ...


## Show the most popular songs in the dataset

In [ ]:
song_data['song'].show()

Materializing SArray

In [ ]:
song_data['listen_count']

dtype: int
Rows: 1116609
[1, 2, 1, 1, 1, 5, 1, 1, 1, 1, 1, 5, 1, 1, 5, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 8, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 5, 5, 2, 1, 9, 2, 2, 1, 1, 6, 2, 1, 10, 1, 1, 9, 1, 1, 1, 10, 2, 6, 13, 6, 4, 2, 4, 1, 3, 5, 1, 8, 18, 12, 2, 2, 1, 2, 2, 2, 1, 3, 4, 3, 1, 2, ... ]

In [ ]:
song_data_listen_counts = song_data.groupby('artist', operations={'total_count': turicreate.aggregate.SUM('listen_count')}).sort('total_count',ascending=False)
song_data_listen_counts[-1]

{'artist': 'William Tabbert', 'total_count': 14}

In [ ]:
kany_west = song_data[song_data['artist'] == 'Kanye West']
len(kany_west['user_id'].unique())

2522

In [ ]:
Foo_Fighters = song_data[song_data['artist'] == 'Foo Fighters']
len(Foo_Fighters['user_id'].unique())

2055

In [ ]:
Taylor_Swift = song_data[song_data['artist'] == 'Taylor Swift']
len(Taylor_Swift['user_id'].unique())

3246

In [ ]:
Lady_GaGa = song_data[song_data['artist'] == 'Lady GaGa']
len(Lady_GaGa['user_id'].unique())

2928

# Count the number of unique users in the data

In [ ]:
users = song_data['user_id'].unique()

In [ ]:
len(users)

66346

# Create a song recommender

In [ ]:
train_data,test_data = song_data.random_split(.8,seed=0)

## Create a very simple popularity recommender

In [ ]:
popularity_model = turicreate.popularity_recommender.create(train_data,
                                                           user_id = 'user_id',
                                                           item_id = 'song')

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 893580 observations with 66085 users and 9952 items.

Data prepared in: 1.76553s

893580 observations to process; with 9952 unique items.

## Use the popularity model to make some predictions

In [ ]:
popularity_model.recommend(users=[users[0]])

user_id,song,score,rank
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Sehr kosmisch - Harmonia,4754.0,1
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Undo - Björk,4227.0,2
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,You're The One - DwightYoakam ...,3781.0,3
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Dog Days Are Over (RadioEdit) - Florence + The ...,3633.0,4
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Revelry - Kings Of Leon,3527.0,5
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Horn Concerto No. 4 in Eflat K495: II. Romance ...,3161.0,6
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Secrets - OneRepublic,3148.0,7
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Fireflies - CharttraxxKaraoke ...,2532.0,8
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Tive Sim - Cartola,2521.0,9
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Drop The World - LilWayne / Eminem ...,2053.0,10


In [ ]:
popularity_model.recommend(users=[users[1]])

user_id,song,score,rank
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Sehr kosmisch - Harmonia,4754.0,1
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Undo - Björk,4227.0,2
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,You're The One - DwightYoakam ...,3781.0,3
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Dog Days Are Over (RadioEdit) - Florence + The ...,3633.0,4
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Revelry - Kings Of Leon,3527.0,5
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Horn Concerto No. 4 in Eflat K495: II. Romance ...,3161.0,6
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Secrets - OneRepublic,3148.0,7
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Hey_ Soul Sister - Train,2538.0,8
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Fireflies - CharttraxxKaraoke ...,2532.0,9
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Tive Sim - Cartola,2521.0,10


# Build a recommender with personalization

In [ ]:
personalized_model = turicreate.item_similarity_recommender.create(train_data,
                                                                  user_id = 'user_id',
                                                                  item_id = 'song')

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 893580 observations with 66085 users and 9952 items.

...

| 2.13s                               | 100              | 9952            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 2.2134s

In [ ]:
subset_test_users = test_data['user_id'].unique()[0:10000]

In [ ]:
personalized_model_table = personalized_model.recommend(subset_test_users,k=1)

recommendations finished on 1000/10000 queries. users per second: 3253.04

recommendations finished on 2000/10000 queries. users per second: 3221.77

recommendations finished on 3000/10000 queries. users per second: 3220.92

recommendations finished on 4000/10000 queries. users per second: 3195.23

recommendations finished on 5000/10000 queries. users per second: 3190.72

recommendations finished on 6000/10000 queries. users per second: 3164.59

recommendations finished on 7000/10000 queries. users per second: 3167.54

recommendations finished on 8000/10000 queries. users per second: 3176.49

recommendations finished on 9000/10000 queries. users per second: 3181.21

recommendations finished on 10000/10000 queries. users per second: 3164.27

In [ ]:
personalized_model_table.groupby('song',operations={'count': turicreate.aggregate.COUNT()}).sort('count',ascending=False)

song,count
Undo - Björk,427
Secrets - OneRepublic,398
Revelry - Kings Of Leon,230
You're The One - DwightYoakam ...,154
Hey_ Soul Sister - Train,114
Sehr kosmisch - Harmonia,111
Fireflies - CharttraxxKaraoke ...,103
Horn Concerto No. 4 in Eflat K495: II. Romance ...,90
OMG - Usher featuringwill.i.am ...,70
The Scientist - Coldplay,50


## Apply personalized model to make song recommendations

In [ ]:
personalized_model.recommend(users=[users[0]])

user_id,song,score,rank
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Cuando Pase El Temblor -Soda Stereo ...,0.019450453611520622,1
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Fireflies - CharttraxxKaraoke ...,0.014473731701190654,2
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Love Is A Losing Game -Amy Winehouse ...,0.014286596041459303,3
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Marry Me - Train,0.014133471709031325,4
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Secrets - OneRepublic,0.013591665488023024,5
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Sehr kosmisch - Harmonia,0.013398789442502536,6
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Te Hacen Falta Vitaminas- Soda Stereo ...,0.01293028317964994,7
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,OMG - Usher featuringwill.i.am ...,0.012777828253232516,8
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Y solo se me ocurreamarte (Unplugged) - ...,0.012341127945826603,9
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,No Dejes Que... -Caifanes ...,0.012104249917543851,10


In [ ]:
personalized_model.recommend(users=[users[1]])

user_id,song,score,rank
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Where The Boat LeavesFrom (Album) - Zac Brown ...,0.06153608858585358,1
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Different Kind Of Fine(Album) - Zac Brown Band ...,0.06052837520837784,2
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Jolene (Album) - ZacBrown Band ...,0.05786820501089096,3
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Sic 'Em On A Chicken(Album) - Zac Brown Band ...,0.05518669635057449,4
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Who's Kissing You Tonight- Jason Aldean ...,0.05306333303451538,5
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,What Country Is - LukeBryan ...,0.037490807473659515,6
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Highway 20 Ride (Album) -Zac Brown Band ...,0.03733157366514206,7
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Do I - Luke Bryan,0.033077359199523926,8
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,One Fine Wire - ColbieCaillat ...,0.03125,9
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Midnight Bottle - ColbieCaillat ...,0.030737705528736115,10


# Apply model to find similar songs in the data set

In [ ]:
personalized_model.get_similar_items(['With Or Without You - U2'])

song,similar,score,rank
With Or Without You - U2,I Still Haven't FoundWhat I'm Looking For ...,0.04285717010498047,1
With Or Without You - U2,Hold Me_ Thrill Me_ KissMe_ Kill Me - U2 ...,0.033734917640686035,2
With Or Without You - U2,Window In The Skies - U2,0.03283584117889404,3
With Or Without You - U2,Vertigo - U2,0.03007519245147705,4
With Or Without You - U2,Sunday Bloody Sunday - U2,0.02713179588317871,5
With Or Without You - U2,Bad - U2,0.02517986297607422,6
With Or Without You - U2,A Day Without Me - U2,0.023715436458587646,7
With Or Without You - U2,Another Time AnotherPlace - U2 ...,0.020325183868408203,8
With Or Without You - U2,Walk On - U2,0.020202040672302246,9
With Or Without You - U2,Get On Your Boots - U2,0.019685029983520508,10


In [ ]:
personalized_model.get_similar_items(['Chan Chan (Live) - Buena Vista Social Club'])

song,similar,score,rank
Chan Chan (Live) - BuenaVista Social Club ...,Murmullo - Buena VistaSocial Club ...,0.1881188154220581,1
Chan Chan (Live) - BuenaVista Social Club ...,La Bayamesa - Buena VistaSocial Club ...,0.1871921420097351,2
Chan Chan (Live) - BuenaVista Social Club ...,Amor de Loca Juventud -Buena Vista Social Club ...,0.18483412265777588,3
Chan Chan (Live) - BuenaVista Social Club ...,Diferente - Gotan Project,0.021459221839904785,4
Chan Chan (Live) - BuenaVista Social Club ...,Mistica - Orishas,0.020576119422912598,5
Chan Chan (Live) - BuenaVista Social Club ...,Hotel California - GipsyKings ...,0.019304990768432617,6
Chan Chan (Live) - BuenaVista Social Club ...,Nací Orishas - Orishas,0.019157111644744873,7
Chan Chan (Live) - BuenaVista Social Club ...,Gitana - Willie Colon,0.018796980381011963,8
Chan Chan (Live) - BuenaVista Social Club ...,Le Moulin - Yann Tiersen,0.018796980381011963,9
Chan Chan (Live) - BuenaVista Social Club ...,Criminal - Gotan Project,0.018779337406158447,10


# Compare the models quantitatively
We now formally compare the popularity and the personalized models using precision-recall curves. 

In [ ]:
model_performance = turicreate.recommender.util.compare_models(test_data, [popularity_model, personalized_model], user_sample=.05)

compare_models: using 2931 users to estimate model performance
PROGRESS: Evaluate model M0


recommendations finished on 1000/2931 queries. users per second: 2878.06

recommendations finished on 2000/2931 queries. users per second: 2887.84


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.023882633913340147 | 0.005493132668158254 |
|   2    | 0.023200272944387566 | 0.010979122852203706 |
|   3    | 0.020584555896736044 | 0.014985289550284437 |
|   4    | 0.019447287615148405 | 0.018461124909436088 |
|   5    | 0.019174343227567402 | 0.023438753709992204 |
|   6    | 0.018310019333560752 | 0.027243055288091133 |
|   7    | 0.01744894477750163  | 0.03068575345955082  |
|   8    | 0.01646195837598088  | 0.03270089685135746  |
|   9    | 0.015391030744152576 | 0.03515808559309072  |
|   10   | 0.014773114977823304 | 0.037839601734176956 |
+--------+----------------------+----------------------+
[10 rows x 3 columns]

PROGRESS: Evaluate model M1


recommendations finished on 1000/2931 queries. users per second: 2680.48

recommendations finished on 2000/2931 queries. users per second: 2686.4


Precision and recall summary statistics by cutoff
+--------+---------------------+---------------------+
| cutoff |    mean_precision   |     mean_recall     |
+--------+---------------------+---------------------+
|   1    | 0.14193108154213568 | 0.04020885763465092 |
|   2    | 0.11992494029341512 | 0.06381147663133338 |
|   3    | 0.10462868190606164 | 0.08062717495931405 |
|   4    | 0.09433640395769366 | 0.09449082041200774 |
|   5    | 0.08563630160354807 | 0.10544427472523679 |
|   6    |  0.0792675992266575 | 0.11670665615138479 |
|   7    | 0.07379246478529992 | 0.12491230811343401 |
|   8    | 0.06908904810644836 | 0.13344938355173755 |
|   9    | 0.06436938473785976 | 0.13965820751900576 |
|   10   | 0.06069600818833156 | 0.14526072604373505 |
+--------+---------------------+---------------------+
[10 rows x 3 columns]



The table shows that the personalized model provides much better performance.